In [61]:
import pandas as pd
import re
import difflib
from nltk import ngrams
from langdetect import detect, DetectorFactory
from difflib import get_close_matches as gcm

In [65]:
df_skills = pd.read_csv('skills_db/skill.csv')
sk_list = df_skills['skill'].tolist()

def extract_skills(info):
    # Remove ordered list with alphabets: a), b), c),...
    words = re.sub(r'[\s\t\n]+[a-zA-Z\s*]\)+', ' ', info)
    words = re.sub('[\n|,|.|/|\(|\)]', ' ', words).lower().split()
    bigrams = [' '.join(g) for g in ngrams(words, 2)]
    trigrams = [' '.join(g) for g in ngrams(words, 3)]
    results = []
    for skill in sk_list:
        s = skill.lower()
        if '(' in s:
            abb = s[s.find("(")+1:s.find(")")]
            if len(gcm(abb, words, cutoff=0.95)) > 0:
                results.append(skill)
                continue
            s = re.sub(r"[\(].*?[\)]", "", s)
        s2 = s.split()
        if len(s2) == 1:
            if len(gcm(s, words, cutoff=0.9)) > 0:
                results.append(skill)
        elif len(s2) == 2:
            if len(gcm(s, bigrams, cutoff=0.9)) > 0:
                results.append(skill)
        elif len(s2) == 3:
            if len(gcm(s, trigrams, cutoff=0.85)) > 0:
                results.append(skill)
        else:
            if len(gcm(s, trigrams, cutoff=0.8)) > 0:
                results.append(skill)
    return results

def test_skill(skill, info, c1=0.9, c2=0.85, c3=0.8):
    words = re.sub('[\n|,|.|/|\(|\)]', ' ', info).lower().split()
    bigrams = [' '.join(g) for g in ngrams(words, 2)]
    trigrams = [' '.join(g) for g in ngrams(words, 3)]
    results = []
    s = skill.lower()
    print('Unigram: {}'.format(gcm(s, words, cutoff=c1)))
    print('Bigram: {}'.format(gcm(s, bigrams, cutoff=c2)))
    print('Trigram: {}'.format(gcm(s, trigrams, cutoff=c3)))

In [31]:
df = pd.read_csv('myfuturejobs-insights/myfuturejobs_clean.csv')
df.head()

,job_title,job_details,job_type,education,language,has_skills,skills_required,no_skills_required
0,SALES ASSOCIATE,* Under supervision and perform duties to prov...,Permanent,4 - STPM / A Level,English,True,"['Sales', 'Operations', 'Process', 'Service', ...",6
1,SIGN HOUSE ADVERTISING,Kelebihan diberi kepada yang mempunyai kemahir...,Permanent,6 - Bachelor's,Others,True,"['Adobe', 'Advertising']",2
2,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...",Permanent,5 - Diploma / DVM,English,True,"['Operations', 'Distribution']",2
3,Petugas PDK,"1. Bertanggungjawab kepada penyelia PDK, jawat...",Permanent,6 - Bachelor's,Others,False,[],0
4,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...",Permanent,3 - SPM / O Level / SKM,English,True,"['Operations', 'Distribution']",2


In [32]:
df2 = pd.read_csv('myfuturejobs-insights/myfuturejobs_skills4.csv')
df2.head()

,index,title,job_details,job_type,education,job_language,company,company_details,skills,no_skills,job_skills,no_job_skills,ignore_skills,no_ignore_skills
0,0,SALES ASSOCIATE,* Under supervision and perform duties to prov...,Permanent,4 - STPM / A Level,English,GLOBAL PSYTECH SDN BHD\n|\nCyberjaya,"Retail trade, except of motor vehicles and mot...","['Operations', 'Sales', 'Process', 'Service', ...",6,Sales; Selection,2.0,Operations; Process; Service; Product,4.0
1,1,SIGN HOUSE ADVERTISING,Kelebihan diberi kepada yang mempunyai kemahir...,Permanent,6 - Bachelor's,Others,MR DIY (M) SDN BHD\n|\nSeremban,"Advertising and market research, Printing and ...","['Adobe', 'Adobe Illustrator', 'Adobe Photosho...",4,Adobe Illustrator; Adobe Photoshop; Advertising,3.0,Adobe,1.0
2,2,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...",Permanent,5 - Diploma / DVM,English,INTER-EXCEL ADVISORY SDN BHD\n|\nKuala Lumpur,"Retail trade, except of motor vehicles and mot...","['Operations', 'Distribution']",2,Distribution,1.0,Operations,1.0
3,3,Petugas PDK,"1. Bertanggungjawab kepada penyelia PDK, jawat...",Permanent,6 - Bachelor's,Others,IOI PLANTATION SERVICES SDN BHD\n|\nPutrajaya,"Education, Office administrative, office suppo...",[],0,NaN,0.0,NaN,0.0
4,4,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...",Permanent,3 - SPM / O Level / SKM,English,QL Ansan Poultry Farm Sdn Bhd\n|\nPadang Serai,"Retail trade, except of motor vehicles and mot...","['Operations', 'Distribution']",2,Distribution,1.0,Operations,1.0


In [103]:
df2.loc[df2['job_skills'].str.contains('B')]['job_details'].unique().tolist()[0]

"• Strategize and develop the branding and storytelling strategy of the company brand.\n• Strategize and plan marketing performance across multiple marketing channels (online and offline) i.e. website, social media, influencer program and public relation.\n• Strategize, develop and implement a strategic and creative marketing and promotional plan.\n• Develop the overall company brand exposure and portfolio.\n• Manage the retail outlet branding.\n• Work closely with each department to ensure the marketing initiatives are aligned with the company's visions and core values.\n• Track, analyse and review the performance of marketing activities, manage marketing budget and ensure that all marketing material is in line with our brand identity and overall business objectives are achieved.\n• Conduct market research to understand the market situation and the latest consumer trend.\n• Conduct competitor analysis for an in-depth understanding of the company.\n• Lead and develop multifunctional, h

In [104]:
df2.loc[df2['job_skills'].str.contains('D')]['job_details'].unique().tolist()[1]

'-Drafting,\n-Do office work such as admin work, prepare documentation/claim, \n-Do filing work '

In [109]:
df2.loc[df2['job_skills'].str.contains('B')]['job_details'].unique().tolist()[2]

"- Recording transactions such as income and outgoings, and posting them to various accounts.\n- Processing payments.\n- Conducting daily banking activities\n- coordinate  staffs' claim activities"

In [116]:
test_skill('B', "Sdn Bhd - Recording transactions such as income and outgoings, and posting them to various accounts.\n- Processing payments.\n- Conducting daily banking activities\n- coordinate  staffs' claim activities")

Unigram: []
Bigram: []
Trigram: []


In [101]:
df2.loc[df2['job_skills'].str.contains('Distribution')]['job_details'].unique().tolist()[3]

"Job Descriptions:\n\n• Formulate and execute company's digital marketing strategies\n• Build relationships through Online social media channels (Facebook, Linkedin, Google, Instagram etc).\n• Drive online traffic through SEO, Website analytics, Google analytics, content marketing, marketing campaigns.\n• Monitor, analyse with statistics, and suggest ideas to optimise various marketing channels.\n• Responsible to attract potential leads from social media to support the sales team to achieve higher sales closing rate.\n• Plan and monitor the ongoing company presence on social media.\n• Launch optimized online advertisements through Google Adwords, Facebook etc. to increase company and brand awareness.\n• Prepare online contents, promotional write up, collaborate with designer to come out with marketing contents and organize their distribution through various channels\n• Provide creative ideas and prepare online contents and update website\n• Maintain partnerships with media agencies and

In [102]:
df2.loc[df2['job_skills'].str.contains('Distribution')]['job_details'].unique().tolist()[4]

'Scope\n\n- Avid social media geek: Facebook, Website, Instagram, Twitter, Youtube, Tik Tok, Telegram etc\n- Skills in Facebook Ads, Website, Email Marketing/ Google Analytics/ SEO/ SEM/ Growth Hacking/ - Split testing etc would be a bonus.\n- Positive attitude, passionate on collaborative teamwork, excel at interpersonal skills.\n- Excellent written and verbal communication skills.\n- Adaptable to change, able to pivot and find alternate solutions, demonstrated capability in meeting new challenges.\n- Good problem solving skills, able to navigate unexpected situations or conditions.\n- Critical observer-thinker, bringing fresh perspective and offer intuitive solutions and ideas.\n- Demonstrated leadership skills in driving teamwork excellence, conflict resolution.\n- Fluent in English, Bahasa Malaysia, optionally Mandarin.\n- Business-minded where applicable with entrepreneurship skill to ensure business success.\n- Have strong knowledge of applicable system automation, business solut

In [44]:
df2['job_skills'] = df2['job_skills'].fillna('')
df2['ignore_skills'] = df2['ignore_skills'].fillna('')
df_temp = df2.loc[df2['job_skills'].str.contains('D;')]
df_temp.head()

,index,title,job_details,job_type,education,job_language,company,company_details,skills,no_skills,job_skills,no_job_skills,ignore_skills,no_ignore_skills
159,159,Sales Assistant,~5 or 6 working days per week\n~MALE candidate...,Permanent,5 - Diploma / DVM,English,PASARAYA SONGMART (KLUANG) SDN BHD\n|\nKluang,"Manufacture of computer, electronic and optica...","['Communication', 'Sales', 'D', 'English', 'Tr...",5,Communication; Sales; D; English; Training,5.0,,0.0
165,165,Sing Kong Optical & Contact Lens Centre,~5 or 6 working days per week\n~MALE candidate...,Permanent,5 - Diploma / DVM,English,ELUMINAS ENGINEERING SDN BHD\n|\nSenai,"Manufacture of computer, electronic and optica...","['Communication', 'D', 'English', 'Training']",4,Communication; D; English; Training,4.0,,0.0
181,181,Respack Manufacturing Sdn Bhd,1. Position Objective:\na) To achieve sales ta...,Contract,6 - Bachelor's,English,UMI KARIPAP PUSING\n|\nPadang Rengas,Manufacture of rubber and plastics products\n5...,"['Business', 'Leadership', 'Management', 'Mark...",25,Leadership; Management; Marketing; Sales; Clos...,12.0,Business; Planning; Support; B; C; Market; Ser...,13.0
199,199,Marketing manager,1. Position Objective:\na) To achieve sales ta...,Permanent,3 - SPM / O Level / SKM,English,MH ECO SDN BHD\n|\nKluang,Manufacture of rubber and plastics products\n5...,"['Business', 'Leadership', 'Management', 'Mark...",24,Leadership; Management; Marketing; Sales; Clos...,11.0,Business; Planning; Support; B; C; Market; Ser...,13.0
229,229,Respack Manufacturing Sdn Bhd,Position: Head of Quality Assurance\nDepartmen...,Permanent,3 - SPM / O Level / SKM,English,FLEXXO TRADIND SDN BHD\n|\nKuching,Manufacture of rubber and plastics products\n5...,"['Management', 'B', 'C', 'D', 'Quality Assuran...",12,Management; B; D; Quality Assurance; Quality C...,9.0,C; Production; Product,3.0


In [34]:
df_temp['job_details'].iloc[1]

'~5 or 6 working days per week\n~MALE candidate have Driving License D\n~must possess at least SPM qualification with pleasant personality\n~English,Bahasa Malaysia & Mandarin and able to converse with customer at ease\n~Good interpersonal and communication skills\n~Fresh graduates are encouraged to apply\n~Provide training\n~well benefits provided'

In [35]:
df_temp['job_details'].iloc[3]

'1. Position Objective:\na) To achieve sales target and profit margin while creating a positive brand presence in the market\nb) Effective people management towards Company’s objectives. \n\n2. Key Responsibilities & Accountabilities:\na) Answer customers’ questions about products, prices, availability, product uses and credit terms.\nb) Consult with clients after sales to resolve problems and provide on-going support.\nc) Contact regular and prospective customers to demonstrate products, explain product features and solicit orders. \nd) Quote prices, credit or contract terms and delivery dates. \ne) Identify prospective customers by using business directories, following leads from existing clients and suppliers, and other creative sources. \nf) Monitor market conditions, product innovations and competitors’ products, prices and sales.\ng) Recommend products to customers, based on customers’ needs and interest.\nh) Liaise with Production Dept and Sales Coordinator/Customer Service to c

In [41]:
extract_skills(df_temp['job_details'].iloc[3])

['Business',
 'Leadership',
 'Management',
 'Marketing',
 'Planning',
 'Sales',
 'Support',
 'Closing',
 'Ethics',
 'Innovation',
 'Market',
 'Presentation',
 'Service',
 'Customer Service',
 'Customer Service System',
 'Production',
 'Goods',
 'Interest',
 'Price',
 'Product',
 'Profit']

In [36]:
df_temp['job_details'].iloc[5]

'Position: Head of Quality Assurance\nDepartment: Quality Assurance\nReporting Line: Managing Director\n\n1. Position Objective:\na) Responsible for the overall management of QA Dept to ensure optimum product quality towards maximum customer satisfaction.\nb) Effective people management.\nc) Develop and implement quality control and assurance strategies to improve company performance\n\n2. Key responsibilities & Accountabilities:\n\na) Develop and motivate staff to achieve their performance. \nb) Liaise with production personnel and other staffs on routine quality issues to solve problems, seek remedial and corrective action.\nc) Manage vendor quality assurance program to enhance vendor performance.\nd) Contribute ideas, knowledge and experience to improve quality\ne) Generate quality report to track trend and patterns of product quality. \nf) Conduct audits on quality systems to ensure compliance. \ng) Ensure quality systems are well maintained\nh) Liaise with all auditors (internal, 

In [28]:
df_temp = df2.loc[df2['job_skills'].str.contains('B;')]
df_temp.head()

,index,title,job_details,job_type,education,job_language,company,company_details,skills,no_skills,job_skills,no_job_skills,ignore_skills,no_ignore_skills
87,87,QA Officer,Position: QA Officer\nDepartment: Quality Assu...,Permanent,6 - Bachelor's,English,Unichamps marketing company\n|\nKuching,Manufacture of rubber and plastics products\n5...,"['Management', 'B', 'C', 'D', 'Documentation',...",15,B; Documentation; Quality Assurance; Quality C...,9.0,Management; C; D; Email; Production; Product,6.0
91,91,Respack Manufacturing Sdn Bhd,Position: QA Officer\nDepartment: Quality Assu...,Permanent,3 - SPM / O Level / SKM,English,NASI KANDAR JAMAL MOHAMED SDN BHD\n|\nRawang,Manufacture of rubber and plastics products\n5...,"['Management', 'B', 'C', 'D', 'Documentation',...",16,B; Documentation; Quality Assurance; Quality C...,10.0,Management; C; D; Email; Production; Product,6.0
229,229,Respack Manufacturing Sdn Bhd,Position: Head of Quality Assurance\nDepartmen...,Permanent,3 - SPM / O Level / SKM,English,FLEXXO TRADIND SDN BHD\n|\nKuching,Manufacture of rubber and plastics products\n5...,"['Management', 'B', 'C', 'D', 'Quality Assuran...",12,Management; B; D; Quality Assurance; Quality C...,9.0,C; Production; Product,3.0
239,239,QA Manager,Position: Head of Quality Assurance\nDepartmen...,Permanent,3 - SPM / O Level / SKM,English,Protenga Sdn Bhd\n|\nSenai,Manufacture of rubber and plastics products\n5...,"['Management', 'B', 'C', 'D', 'Quality Assuran...",11,Management; B; D; Quality Assurance; Quality C...,8.0,C; Production; Product,3.0
484,484,HINOKI BEVERAGES SDN. BHD.,"Responsible for sourcing, selecting and negoti...",Permanent,6 - Bachelor's,English,Hotouch Sdn Bhd\n|\nJohor Bahru,Manufacture of beverages\n50-250 employees\nSe...,"['Management', 'B', 'C', 'D', 'Negotiation', '...",7,Management; B; C; D; Negotiation,5.0,Goods; Price,2.0


In [45]:
df2.loc[df2['ignore_skills'].str.contains('Microsoft;')]

,index,title,job_details,job_type,education,job_language,company,company_details,skills,no_skills,job_skills,no_job_skills,ignore_skills,no_ignore_skills
358,358,HR and Admin Executives,POSITION : HR & ADMIN EXECUTIVE - Based in Sub...,Permanent,5 - Diploma / DVM,English,kamlax global technologies sdnbhd\n|\nCyberjaya,"Other professional, scientific and technical a...","['Communication', 'Development', 'Management',...",22,Security; Effective Communication; Microsoft O...,9.0,Communication; Development; Management; Operat...,13.0
360,360,MICRO-CTL AUTOMATION SDN BHD,POSITION : HR & ADMIN EXECUTIVE - Based in Sub...,Permanent,2 - PMR / PT3,English,Agrox Holdings Sdn Bhd\n|\nPuchong,"Other professional, scientific and technical a...","['Communication', 'Development', 'Management',...",23,Security; Effective Communication; Microsoft O...,10.0,Communication; Development; Management; Operat...,13.0
1183,1183,Accounts & Administrative Assistant,Job Responsibilities:\n1. Preparation of mont...,Contract,3 - SPM / O Level / SKM,English,iPrice Group Sdn Bhd\n|\nKuala Lumpur,Other manufacturing\n10-50 employees\nKuala Lu...,"['Business', 'Accountancy', 'Accounting', 'Fin...",14,Accountancy; Finance; Management; Accounting S...,8.0,Business; Accounting; Email; Data; Microsoft; ...,6.0
1197,1197,NEW LEVIS TRADING SDN BHD,Job Responsibilities:\n1. Preparation of mont...,Permanent,3 - SPM / O Level / SKM,English,iPrice Group Sdn Bhd\n|\nKuala Lumpur,Other manufacturing\n10-50 employees\nKuala Lu...,"['Business', 'Accountancy', 'Accounting', 'Fin...",13,Accountancy; Finance; Management; Accounting S...,7.0,Business; Accounting; Email; Data; Microsoft; ...,6.0
1198,1198,Business Internship,Job Responsibilities:\n1. Preparation of mont...,Permanent,3 - SPM / O Level / SKM,English,THB INFRA SDN BHD\n|\nAlor Setar,Other manufacturing\n10-50 employees\nKuala Lu...,"['Business', 'Accountancy', 'Accounting', 'Fin...",13,Accountancy; Finance; Management; Accounting S...,7.0,Business; Accounting; Email; Data; Microsoft; ...,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19115,19115,QUBE MEDICAL PRODUCTS SDN. BHD.,Description :\nKPMG Corporate Practice offers ...,Permanent,3 - SPM / O Level / SKM,English,PEMBINAAN EASTERN ALUMINIUM SDN BHD\n|\nJohor ...,"Financial service activities, except insurance...","['Business', 'Accounting', 'Communication', 'F...",13,Accounting; Communication; Finance; Writing; M...,7.0,Business; Application; Microsoft; Service; Pro...,6.0
19116,19116,Mould Designer,Description :\nKPMG Corporate Practice offers ...,Permanent,6 - Bachelor's,English,AGENSI PEKERJAAN CAREERMASTER SDN BHD\n|\nJoho...,"Financial service activities, except insurance...","['Business', 'Accounting', 'Communication', 'F...",12,Accounting; Communication; Finance; Writing; M...,7.0,Business; Application; Microsoft; Service; Tax,5.0
19117,19117,DAUN EMAS CONSULTANCY,Description :\nKPMG Corporate Practice offers ...,Internship,5 - Diploma / DVM,English,AIRASIA BERHAD\n|\nKLIA,"Financial service activities, except insurance...","['Business', 'Accounting', 'Communication', 'F...",12,Accounting; Communication; Finance; Writing; M...,7.0,Business; Application; Microsoft; Service; Tax,5.0
19419,19419,BIG SUPERMART SDN BHD,Job Description\nDuties & Responsibilities :\n...,Contract,5 - Diploma / DVM,English,GRASS2ROUTE SDN BHD\n|\nCheras,"Retail trade, except of motor vehicles and mot...","['Business', 'Management', 'Operations', 'Supp...",19,Management; Ethics; Negotiation; English; Micr...,9.0,Business; Operations; Support; Email; Language...,10.0


In [66]:
skill_tree = pd.read_csv('skills_db/skill_tree.csv')
df_st = skill_tree.replace(df_skills.set_index('skill_id').to_dict()['skill'])\
    .rename(columns={'skill_1_id': 'skill', 'skill_2_id': 'parent'})
df_st.head()

,skill,parent
0,Accountancy,Business
1,Accounting,Business
2,Aerospace Engineering,Engineering
3,Agricultural Engineering,Engineering
4,Agronomy,Applied Science


In [83]:
df_st.loc[df_st['skill'] == 'Python', 'parent'].tolist()

['Business Analytics', 'Programming Language']

In [84]:
def get_relationship(skill):
    ship = [skill]
    get_ship(skill, ship, [])
    return ship

def get_ship(skill, ship, checked):
    parents = df_st.loc[df_st['skill'] == skill, 'parent'].tolist()
    ship.append(parents)
    if len(parents) != 0:
        for parent in parents:
            if parent not in checked:
                checked.append(parent)
                get_ship(parent, ship, checked)

In [89]:
get_relationship('Management')

['Management', ['Applied Science', 'Business', 'Social Science'], [], [], []]

In [74]:
def get_main_domain(skill):
    domains = []
    get_domain(skill, domains, [])
    return list(set(domains))

def get_domain(skill, domains, checked):
    parents = df_st.loc[df_st['skill'] == skill, 'parent']
    if len(parents) == 0:
        domains.append(skill)
    else:
        for parent in parents:
            if parent not in checked:
                checked.append(parent)
                get_domain(parent, domains, checked)

In [76]:
get_main_domain('Marketing')

['Applied Science', 'Social Science', 'Business']

In [12]:
df_st.loc[df_st['skill'] == 'Bag']

,skill,parent
224,Bag,Discrete Mathematics


In [13]:
df_st.loc[df_st['skill'] == 'Box']

,skill,parent
1500,Box,Cloud Application
1501,Box,Platform as a Service (PaaS)


In [17]:
test_skill('D', 'd')

Unigram: ['d']
Bigram: []
Trigram: []


In [128]:
N_TRAIN = 20
dict_list = []

for i in range(N_TRAIN):
    row = df.iloc[i]
    all_info = row['job_title'] + ' ' + row['job_details']
    skills = extract_skills(all_info)
    ignore_skills = []
    for j, skill in enumerate(skills):
        if any(skill in string for string in skills[:j] + skills[j+1:]):
            ignore_skills.append(skill)
    return_skills = [s for s in skills if s not in ignore_skills]
    dict_list.append({
        'job_title': row['job_title'],
        'job_details': row['job_details'],
        'skills_required': return_skills,
        'no_skills_required': len(return_skills),
        'ignored_skills': ignore_skills
    })

In [129]:
df2 = pd.DataFrame.from_dict(dict_list)
df2

,job_title,job_details,skills_required,no_skills_required,ignored_skills
0,SALES ASSOCIATE,* Under supervision and perform duties to prov...,"[Sales, Operations, Process, Service, Selectio...",6,[]
1,SIGN HOUSE ADVERTISING,Kelebihan diberi kepada yang mempunyai kemahir...,"[Adobe Illustrator, Adobe Photoshop, Advertising]",3,[Adobe]
2,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...","[Operations, Distribution]",2,[]
3,Petugas PDK,"1. Bertanggungjawab kepada penyelia PDK, jawat...",[],0,[]
4,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...","[Operations, Distribution]",2,[]
5,Vacancy For Technician,Engineering assistants ensure the administrati...,"[Engineering, Administration]",2,[]
6,CADET PLANTER,• Assists the Estate Management team in managi...,"[Management, Development, Communication and Se...",16,[Communication]
7,Perunding Setia SAR,"-Drafting,\n-Do office work such as admin work...",[Documentation],1,[]
8,Factory and Warehouse Workers,Factory hands assist machine operators and pro...,"[Make, Product]",2,[]
9,Junior Designer,Kelebihan diberi kepada yang mempunyai kemahir...,"[Adobe Illustrator, Adobe Photoshop]",2,[Adobe]


In [131]:
df_st.loc[df_st['skill'] == 'D']

,skill,parent
331,D,Programming Language


In [94]:
df_st.loc[df_st['skill'].str.contains('Profit')]

,skill,parent
2665,Profit,Microeconomics
2674,Rate of Profit,Profit
